<a href="https://colab.research.google.com/github/KK-Singh333/IITI_BOT/blob/main/Sub_Query_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Radhey Radhey
!pip install pathway pathway[xpack-llm] litellm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of boto3 to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 116.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.3/145.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 83.2 

In [ ]:
!pip install pathway

In [1]:
import pathway as pw
from pathway.xpacks.llm import llms
# from pathway.udfs import ExponentialBackoffRetryStrategy

In [2]:
import os
# from getpass import getpass
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY") or "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"

In [3]:
# Setting model
from pathway.udfs import ExponentialBackoffRetryStrategy
llm = llms.LiteLLMChat(model="groq/meta-llama/llama-4-scout-17b-16e-instruct", retry_strategy=ExponentialBackoffRetryStrategy(max_retries=2), temperature = 0.0)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


In [4]:
system_prompt = """
You are an AI language model assistant.
Your task is to generate five different versions of the given user question to retrieve relevant documents from a vector database.
By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search.
Provide these alternative questions separated by newlines.
Output only the generated queries not including any other text.
"""


messages = pw.debug.table_from_rows(
    schema = pw.schema_from_types(user_id = str, questions=list[dict]),
    rows=[
        (
            "user1",
            [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": "What are the facilities in IIT Indore"},
            ],
        )
    ],
)

In [5]:
responses = messages.select(user_id = pw.this.user_id, result=llm(pw.this.questions))


In [ ]:
pw.debug.compute_and_print(messages)
pw.debug.compute_and_print(responses)

In [6]:
@pw.udf
def split_lines(text: str) -> list[str]:
    return text.splitlines()

In [7]:
split_table = responses.select(
    user_id=pw.this.user_id,
    questions = split_lines(pw.this.result)  # This gives a List[str]
)
response = split_table.flatten(pw.this.questions)
pw.debug.compute_and_print(split_table)
pw.debug.compute_and_print(response)

            | user_id | questions
^X1MXHYY... | user1   | ('What are the campus facilities offered by IIT Indore', 'What are the amenities provided to students at IIT Indore', 'What infrastructure does IIT Indore have for its students', 'What are the recreational facilities available at IIT Indore', 'What kind of support services does IIT Indore offer to its students')
            | questions                                                           | user_id
^X1MQZF8... | What are the amenities provided to students at IIT Indore           | user1
^X1MGYPB... | What are the campus facilities offered by IIT Indore                | user1
^X1MKPG8... | What are the recreational facilities available at IIT Indore        | user1
^X1MZC0N... | What infrastructure does IIT Indore have for its students           | user1
^X1MYJYZ... | What kind of support services does IIT Indore offer to its students | user1


In [8]:
# DocStore
import pathway as pw
from pathway.xpacks.llm import embedders

embedder = embedders.SentenceTransformerEmbedder(model="all-MiniLM-L6-v2")

# Create a table with text to embed
t = pw.debug.table_from_rows(
    schema = pw.schema_from_types(doc_id = int, chunks = str),
    rows=[
        (
            1,
            "Facilities1. 24 hrs high-speed internet facility.2. Indoor Sports facilities."
        ),
        (
            2,
            """facilities.3. Badminton Court.4. Common Room Facility.5. Newspaper and Magazine in the common area.6. Laundry Facility.7. Equipped with basic furniture like cot, study table, and an almirah for storage.8. CCTV Surveillance and 24 Hrs. Security Guard in Each Hostel."""
        )
    ],
)

# Extract the embedded text
vector_store = t.with_columns(embedding=embedder(pw.this.chunks))
# pw.debug.compute_and_print(t)
vector_store.typehints()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

mappingproxy({'doc_id': int,
              'chunks': str,
              'embedding': numpy.ndarray[typing.Any, numpy.dtype[typing.Any]]})

In [ ]:
class InputSchema(pw.Schema):
    row_id: str
    chunk: str
    embedding: list[list[float]]
    url: str

vector_store = pw.io.csv.read(
  'embedding_sample.csv',
  schema=InputSchema,
  mode="static"
)

In [ ]:
# Retrieval

# Queries ko embedd kar dete hai
response += response.select(embedding=embedder(pw.this.questions))


In [19]:
from pathway.stdlib.ml.index import KNNIndex

doc_index = KNNIndex(
    vector_store.embedding,
    vector_store,
    n_dimensions= embedder.get_embedding_dimension(),  # dimension for all-MiniLM-L6-v2
    distance_type = "cosine"
    # n_and_d=2
)

results = doc_index.get_nearest_items(
    response.embedding,
    k=1  # top 5 most similar documents
).select(doc_id = pw.this.doc_id, chunks = pw.this.chunks)

/usr/local/lib/python3.11/dist-packages/pathway/internals/column_properties.py:124: UserWarning: Universes in with_universe_of have different value of the append_only property. Setting append_only of the output to True
  warnings.warn(


In [20]:
pw.debug.compute_and_print(results)

            | doc_id | chunks
^X1MYJYZ... | ()     | ()
^X1MZC0N... | ()     | ()
^X1MGYPB... | ()     | ()
^X1MQZF8... | ()     | ()
^X1MKPG8... | (2,)   | ('facilities.3. Badminton Court.4. Common Room Facility.5. Newspaper and Magazine in the common area.6. Laundry Facility.7. Equipped with basic furniture like cot, study table, and an almirah for storage.8. CCTV Surveillance and 24 Hrs. Security Guard in Each Hostel.',)
